# Playground

In [ ]:
from pcap_handler import pcapHandler, dfHandler
import pandas as pd
from datetime import datetime

In [ ]:
pcap2df = pcapHandler(file="data\pcap\capture_2023-02-25_210200.pcap", verbose=True)
df = pcap2df.to_DF(head=True)

In [ ]:
df.head()

In [ ]:
columns = [ 
    'version',
    'ihl',
    'tos',
    'len',
    'id',
    'flags_fragment',
    'frag',
    'ttl',
    'proto',
    'chksum',
    'src',
    'dst',
    'options',
    'time',
    'sport',
    'dport',
    'seq',
    'ack',
    'dataofs',
    'reserved',
    'flags',
    'window',
    'chksum_payload',
    'urgptr',
    'options_payload',
    'payload',
    'payload_raw',
    'payload_hex',
]

In [ ]:
df = df.set_axis(columns, axis=1)

In [ ]:
df.to_excel("data\pcap_handler.xlsx", index=False)

# database

In [ ]:
df = df.convert_dtypes()

# df['payload_hex'] = df['payload_hex'].astype(bytes)
# df['payload_raw'] = df['payload_raw'].astype(bytes)
df['flags_fragment'] = df['flags_fragment'].astype("string")
df['flags'] = df['flags'].astype("string")
df["time"] = pd.to_numeric(df["time"])

df.head()

In [ ]:
df.info()

In [ ]:
engine = create_engine(CONNSTR)

# Write data into the table in sqllite database
try:
  df[:1].to_sql("activity_", engine )
except Exception as e:
  #print("error \t:"+e)
  pass

# extract payload

In [ ]:
records = df.to_dict('records')

In [ ]:
for idx, rec in enumerate(records):
    try:

        payload = str(rec['payload_raw'])
        lst = payload.replace("\\r","").replace("b'","").split("\\n")

        lst = list(elem for elem in lst if elem not in [ "'", '' ])

        for _idx, elem in enumerate(lst):
            if _idx == 0:
                _elem  = lst[0].split(" ")
                records[idx]['payload_method'] = _elem[0]
                records[idx]['payload_route'] = _elem[1]
                records[idx]['payload_protocol'] = _elem[2]
            else:
                kv = elem.split(":")
                records[idx]["payload_"+kv[0]] = kv[1]
        
        
        records[idx]['datetime'] = datetime.fromtimestamp( rec['time'] )

    except Exception as e:
        # print(str(e))
        pass

In [ ]:
df = pd.DataFrame(records)
df.head()

In [ ]:
columns = list(df.columns)
df = df.set_axis(list( col.lower() for col in columns ), axis=1)

In [ ]:
df.columns

In [ ]:
df.to_excel("data\pcap_handler_reformat.xlsx", index=False)